**`ICUBAM`: ICU Bed Availability Monitoring and analysis in the *Grand Est région* of France during the COVID-19 epidemic.**

https://doi.org/10.1101/2020.05.18.20091264

Python notebook for the sir-like modeling (see Section IV.1 of the main paper).

(i) compute model, fit to the data, and save best parameters found via maximum likelihood

In [1]:
import numpy as np
import pandas as pd
import os.path as op
from scipy.optimize import minimize
from collections import namedtuple
import model_icubam as micu

np.random.seed(13)

In [2]:
data_pop = pd.read_csv(op.join('data', 'pop_dep_2020.csv'), delimiter='\t')

#Fig. 17 (left) is realized using first_date = '2020-03-19' and last_date = '2020-04-29'.
#Fig. 11 and Fig. 17 (right) are realized using first_date = '2020-03-19' and last_date = '2020-04-27'.
first_date = '2020-03-19'
last_date = '2020-04-27'

data = pd.read_csv(op.join(micu.data_path, 'all_bedcounts_2020-05-04_11h02.csv'), index_col=0)              
data = data.groupby(['date', 'department']).sum().reset_index()
data = data[data.date >= first_date]
data = data[data.date <= last_date]

In [3]:
sites = ['Ardennes', 'Aube', 'Bas-Rhin', 'Haut-Rhin',
       'Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Vosges']
n_sites = len(sites)

depname2depid = {'Ardennes':8, 'Aube':10, 'Marne':51, 'Haute-Marne':52,
                 'Meurthe-et-Moselle':54, 'Meuse':55, 'Moselle':57, 'Bas-Rhin':67,
                 'Haut-Rhin':68, 'Vosges':88}

In [4]:
micu.make_dir(micu.fig_path)
micu.make_dir(micu.model_path)

In [5]:
# modeling

In [6]:
def compute_llh(data_obs, data_hat):
    cobs, xobs = data_obs
    chat, xhat = data_hat  
    llh = -(np.sum(np.power(cobs-chat, 2))
           + np.sum(np.power(xobs-xhat, 2)))
    return llh

In [7]:
def evaluate_estimation(fun_model, pop, params_init, data_obs):
    n_days = len(data_obs[0])
    data_hat = fun_model(pop, params_init, n_days)
    return -compute_llh(data_obs, data_hat)

In [8]:
def fit_model(fun_model, data, sites, params_init, bounds):
    params_hat = []
    llh = []
    for dep in sites:
        condition = data.department==dep        
        pop = data_pop[data_pop.dep=='{}'.format(depname2depid[dep])]['pop'].values[0]
        n_days = data[condition].date.shape[0]
        data_obs = (data[condition]['n_covid_occ'].values,
                    data[condition]['n_covid_deaths'].values
                    +data[condition]['n_covid_healed'].values)
        optim_fun = lambda x: evaluate_estimation(fun_model, pop, x, data_obs)
        optim_results = minimize(fun=optim_fun, x0=params_init, bounds=bounds)
        params_hat.append(optim_results.x)
        llh.append(-optim_results.fun)
    return np.array(params_hat), np.array(llh)

In [9]:
Params = namedtuple('Params', 'n_days_pre alpha_e beta wei wiout wic wcc wcx')
Bounds = namedtuple('Bounds', 'n_days_pre alpha_e beta wei wiout wic wcc wcx')

compute_model = micu.compute_model_seir

In [10]:
n_days_pre_rg = np.arange(0, 25)

llh_n_days_pre = []
params_hat_n_days_pre = []
for n_days_pre in n_days_pre_rg:
    params_init = Params(n_days_pre=n_days_pre, alpha_e=1e-5, beta=0.01, 
                         wei=0.1, wiout=0.2, wic=0.1, wcc=0.1, wcx=0.5)
    bounds = Bounds(n_days_pre=(n_days_pre, n_days_pre), alpha_e=(0, 1), beta=(0, 1),
                    wei=(0, 1), wiout=(0, 1), wic=(0, 1), wcc=(0, 1), wcx=(0, 1))
    
    params_hat, llh = fit_model(compute_model, data, sites, params_init, bounds)
    llh_n_days_pre.append(llh)
    params_hat_n_days_pre.append(params_hat)

llh_n_days_pre = np.array(llh_n_days_pre)
params_hat_n_days_pre = np.array(params_hat_n_days_pre)

params_hat = []
for k, dep in enumerate(sites):
    ind_best = np.argmax(llh_n_days_pre[:,k])
    params_hat.append(params_hat_n_days_pre[ind_best, k, :])
params_hat = np.array(params_hat)

In [11]:
params_name = list(params_init._asdict().keys())

df_param = pd.DataFrame([dict([(param, params_hat[k,i]) for i, param in enumerate(params_name)] + [('dep',sites[k])]) 
              for k in range(n_sites)])

df_param.to_csv(op.join(micu.model_path, 'params_hat_{}.csv'.format(last_date)))